<a href="https://colab.research.google.com/github/Cseudave/automatic_tops/blob/main/08_Convertir_clips_a_video_corto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Crear los videos cortos

MoviePy es una excelente librería que permite trabajar con videos usando python. Aunque no está optimizada para utilizar tarjetas gráficas, será muy útil.  

## Instalar librerías necesarias

In [ ]:
# Instalamos librerías
!pip install google-api-python-client
!pip install pytube

!pip install moviepy
!pip install imageio-ffmpeg 
!pip install imageio
!apt install imagemagick

!pip install IPython
!pip install mpld3 

!pip install fuzzywuzzy

!pip install playsound

In [ ]:
# Actulizamos librerías
!pip install --upgrade imageio-ffmpeg
!pip install --upgrade moviepy 
!pip install --upgrade imageio
!pip install --upgrade ffmpeg

In [ ]:
# Configuración adicional para trabajar con texto en movipy usando ImageMagick
!cat /etc/ImageMagick-6/policy.xml | sed 's/none/read,write/g'> /etc/ImageMagick-6/policy.xml

In [ ]:
# Importamos librerías
from apiclient.discovery import build
from pytube import YouTube
import os 
import time
from bs4 import BeautifulSoup

import re
import numpy as np
from numpy import arange
import pandas as pd
import matplotlib.pyplot as plt

import csv

from IPython.core.display import HTML
import mpld3 
from scipy.stats import moment
from moviepy.editor import *

import os 

from moviepy.audio.AudioClip import concatenate_audioclips
from moviepy.editor import concatenate_videoclips
from moviepy.editor import VideoFileClip
from moviepy.editor import ColorClip
from moviepy.editor import CompositeVideoClip
from moviepy.editor import *

from IPython.display import Audio
from datetime import datetime

## Clips a video short

In [ ]:
def composicion(clip2, clip1 = 0, t=4, text_top='', text_user='', lista =[]):
  if clip1 == 0:
    clip1 = VideoFileClip('ops_templates/Intro.mp4')
    clip1 = text_intro(clip1, text_top, text_user, lista)
  clip = CompositeVideoClip([clip1, clip2.set_start(clip1.duration - t)])
  return clip

In [ ]:
# Toma la lista de animes para el top y busca los clips
def list_clips(top):
    clips = []
    print(len(top))
    for i in range(len(top) - 1, -1, -1):
      video = VideoFileClip('ops_clips/' + str(top[i]) + '_clip.mp4') 
      #video = text_position(clip,str(i + 1))
      #video = text_number(clip, i + 1)
      video = video.crossfadein(2)
      video = video.crossfadeout(1.5)
      video= video.audio_fadein(2)
      video= video.audio_fadeout(1.5)
      clips.append(video)
    return clips

In [ ]:
# Agrega el número de la posición en cada caso usando plantillas
def text_number(clip, n):
  img = ImageClip('ops_templates/transparented/t' + str(n) + '.png')
  img = img.set_duration(6.0)
  img = img.crossfadein(1)
  img = img.crossfadeout(2)
  canvas = ColorClip((1080, 1920), color=(255, 255, 255))
# clip_fondo = ColorClip((img.shape[0], img.shape[1]), color=(0, 0, 0), duration=10)

  # Crear el lienzo combinando los clips
  final_clip = CompositeVideoClip([
      canvas.subclip(t_start=0, t_end=clip.duration),
      clip,
      img.set_pos((0,0))])
  return final_clip

In [ ]:
#Agrega el intro y la salida según el idioma deseado

def intro_end(top, name_top, name_user, lang='es'):
  
  top = VideoFileClip('tops_hechos/' + str(top) + '.mp4')
  intro = VideoFileClip('ops_templates/Intro.mp4')
  button = VideoFileClip('ops_templates/buttonOK.mp4')

  if lang == 'es':
    salida = VideoFileClip('ops_templates/salida_es.mp4')
  elif lang == 'en':
    salida = VideoFileClip('ops_templates/salida_en.mp4')
  else:
    print('Lenguaje no reconocido')

  intro = intro.crossfadeout(0.5)
  salida = salida.crossfadein(1)
  top = top.crossfadein(1)
  t = top.duration


  if len(name_top) > 20:
    size_top = 110 - (len(name_top) - 20)
  else:
    size_top = 110

  if len(name_user) > 10:
    size_user = 90 - (len(name_user) - 12)
  else:
    size_user = 80


  text_top1 = TextClip(str(name_top), fontsize = size_top, 
                      font='MavenPro-Bold.ttf',
                      stroke_width=2, stroke_color = '#999999',
                      color = '#E6E6E6',
                      method = 'caption', size = (1000, 1920),
                      kerning=-5, interline=-20, align='center') 
  #stroke_color = '#7A9C99'

  text_top1 = text_top1.set_pos((40, -860)).set_duration(t)
  text_top1 = text_top1.set_start(0)
  text_top1 = text_top1.crossfadeout(0.5)

  text_user1 = TextClip(str(name_user), fontsize = size_user, 
                        font='MavenPro-Bold.ttf',
                        stroke_width=2, stroke_color = '#999999',
                        color = '#E6E6E6',
                        method = 'caption', size = (500, 1980),
                        kerning=-5, interline=-20, align='East')
    
  text_user1 = text_user1.set_pos((540, 820)).set_duration(t) 
  text_user1 = text_user1.set_start(0)
  text_user1 = text_user1.crossfadeout(0.5)

  button = button.set_duration(4)
  button = button.crossfadein(0.5)
  button = button.crossfadeout(0.5)


  result = CompositeVideoClip([intro,
                              top.set_start(intro.duration - 1.5),
                              text_top1, text_user1,
                              button.set_start(t//3), button.set_start(2*t//3),
                              salida.set_start(top.duration),
                              button.set_start(t).set_pos((0, 1710))
                              ])
  


  compression_args = {
    'codec': 'libx264',  # Codec de video a utilizar
    'bitrate': '2000k',  # Tasa de bits a utilizar
    'fps': 30,  # Cuadros por segundo
      }

  # result.save_frame('tops_hechos/' + str(name_top) +' - ' 
  #                 + str(name_user) + '_' + str(lang) + '.png', t=0)


  result.write_videofile('tops_hechos/Publicar/' + str(name_top) +' - ' + 
                         str(name_user) +'_' + str(lang) + '.mp4',
                          **compression_args)
  

  top.close()
  intro.close()
  salida.close()
  
  return result

In [ ]:
# Función tentativa para agregar los géneros durante cada clip
def genre(video, generos, score = 0):
  colors = ['#B8BFBF', '#IF8FFD', '#7C0IFD', '#FFCD3C']
  genre_clips = TextClip('Generos:', fontsize = 120*(1 + score/10), 
                        font='MavenPro-Bold.ttf',
                        stroke_width=3, stroke_color = '#EEEEEE',
                        color = colors[score],
                        method = 'caption', size = (864, 1920),
                        kerning=-5, interline=-20, align='East') 
  genre_clips = genre_clips.set_pos((108, -770)).set_duration(0.5)
  genre_clips = genre_clips.crossfadein(0.25)

  for genero in generos:

    text_genre = TextClip(str(genero).upper(), fontsize = 120*(1 + score/10), 
                        font='MavenPro-Bold.ttf',
                        stroke_width=3, stroke_color = '#EEEEEE',
                        color = colors[score],
                        method = 'caption', size = (864, 1920),
                        kerning=-5, interline=-20, align='East') 
    text_genre = text_genre.set_pos((0, -770)).set_duration(1.5)
    text_genre = text_genre.crossfadein(0.5)
    genre_clips =  CompositeVideoClip([genre_clips, text_genre.set_start((genre_clips.duration))])

  intro =  CompositeVideoClip([intro, genre_clips])
  
  return intro

In [ ]:
# Agrega la posición usando texto
def text_position(positions):

  texts_name = []
  texts_shadow = []
  texts_shadow2 = []

  for position in range(positions, -1, -1):
    text_name = TextClip(str(position), fontsize = 600, 
                      font='theater.bold-condensed.ttf',
                      stroke_width=3, stroke_color = '#141414',
                      color = '#D4D4D4',
                      method = 'caption', size = (864, 1920),
                      kerning=-20, interline=-30, align='West')
    

    text_name = text_name.set_pos((55, 730)).set_duration(5)
    text_name = text_name.crossfadein(1) 
    text_name = text_name.crossfadeout(1)

    text_shadow = TextClip(str(position), fontsize = 600, 
                    font='theater.bold-condensed.ttf',
                    stroke_width=0, stroke_color = '#111111',
                    color = '#343434',
                    method = 'caption', size = (864, 1920),
                    kerning=-20, interline=-30, align='West') 

    text_shadow = text_shadow.set_pos((75, 740)).set_duration(5)
    text_shadow = text_shadow.crossfadein(1)
    text_shadow = text_shadow.crossfadeout(1)


    text_shadow2 = TextClip(str(position), fontsize = 600, 
                  font='theater.bold-condensed.ttf',
                  stroke_width=0, stroke_color = '#111111',
                  color = '#141414',
                  method = 'caption', size = (864, 1920),
                  kerning=-20, interline=-30, align='West')

    text_shadow2 = text_shadow2.set_pos((50, 725)).set_duration(5)
    text_shadow2 = text_shadow2.crossfadein(1) 
    text_shadow2 = text_shadow2.crossfadeout(1)
    
    texts_name.append(text_name)
    texts_shadow.append(text_shadow)
    texts_shadow2.append(text_shadow2)   

  return  texts_name, texts_shadow, texts_shadow2

In [ ]:
# Construye el video top con los clips
def clips_top(lista, name='', t=3):

  clips = list_clips(lista)
  names, shadows, shadows2 = text_position(len(lista))

  img = ImageClip('ops_templates/tbase.png')

  img = img.set_duration(5)
  img = img.crossfadein(1)
  img = img.crossfadeout(1)

  time = clips[0].duration - t
  delay = 1.5
  all = []
  for i in range(len(clips)):
    all.append(clips[i].set_start(i*time))
    all.append(shadows[i].set_start(i*time + delay))
    all.append(img.set_start(i*time + delay))
    all.append(shadows2[i].set_start(i*time + delay))
    all.append(names[i].set_start(i*time + delay))
 
  result = CompositeVideoClip(all)

  if name == '':
    now = datetime.now()
    titulo = str(now.strftime("%Y-%m-%d %H:%M:%S")) 
  else:
    titulo = str(name)

  compression_args = {
      'codec': 'libx264',  # Codec de video a utilizar
      'bitrate': '2000k',  # Tasa de bits a utilizar
      'fps': 30,  # Cuadros por segundo
  }


  result.write_videofile('tops_hechos/' + str(titulo) + '.mp4',
                          **compression_args)
  print(str(titulo) + '.mp4')
  for clip in clips:
    clip.close()

  return result